# Hierarchical Quantum Classifier

Code for paper "Hierarchical Quantum Classifier" (2018 Grant et al.). Investigate Tree Tensor Network (TTN) and Multi-scale Entanglement Renormalization Ansatz (MERA) structures.
In this code, we investigate how different unitary ansatze effects the results of the TTN circuit structures.

In [1]:
import pennylane as qml
from pennylane import numpy as np

### 1. Data loading and processing

MNIST data loading and processing procedures.

In [3]:
classes = [0, 1]

In [4]:
import tensorflow as tf
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses

def data_load_and_process(classes = [0,1], feature_reduction = 'resize256', label01 = False):
    
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0 #normalize the data
    
    x_train_filter_01 = np.where((y_train == classes[0]) | (y_train == classes[1]))
    x_test_filter_01 = np.where((y_test == classes[0]) | (y_test == classes[1]))
    
    x_train_01, x_test_01 = x_train[x_train_filter_01], x_test[x_test_filter_01]
    y_train_01, y_test_01 = y_train[x_train_filter_01], y_test[x_test_filter_01]
    
    if label01 == True:
        y_train_01 = [1 if y ==classes[0] else 0 for y in y_train_01]
        y_test_01 = [1 if y ==classes[0] else 0 for y in y_test_01]
    elif label01 == False:
        y_train_01 = [1 if y ==classes[0] else -1 for y in y_train_01]
        y_test_01 = [1 if y ==classes[0] else -1 for y in y_test_01]
        
    
    if feature_reduction == 'resize256':   
        x_train_01 = tf.image.resize(x_train_01[:], (256, 1)).numpy()
        x_test_01 = tf.image.resize(x_test_01[:], (256, 1)).numpy()
        x_train_01, x_test_01 = tf.squeeze(x_train_01), tf.squeeze(x_test_01) 
        return x_train_01, x_test_01, y_train_01, y_test_01
    
    elif feature_reduction == 'pca8':
        x_train_01 = tf.image.resize(x_train_01[:], (784, 1)).numpy()
        x_test_01 = tf.image.resize(x_test_01[:], (784, 1)).numpy()
        x_train_01, x_test_01 = tf.squeeze(x_train_01), tf.squeeze(x_test_01)
        
        pca = PCA(8)
        x_train_01 = pca.fit_transform(x_train_01)
        x_test_01 = pca.transform(x_test_01)
        
        #Rescale for angle embedding
        x_train_01, x_test_01 = (x_train_01 + 10) * (np.pi / 20), (x_test_01 + 10) * (np.pi / 20)
        
        return x_train_01, x_test_01, y_train_01, y_test_01

    elif feature_reduction == 'autoencoder8':
        latent_dim = 8 
        class Autoencoder(Model):
            def __init__(self, latent_dim):
                super(Autoencoder, self).__init__()
                self.latent_dim = latent_dim   
                self.encoder = tf.keras.Sequential([
                layers.Flatten(),
                  layers.Dense(latent_dim, activation='relu'),
                ])
                self.decoder = tf.keras.Sequential([
                layers.Dense(784, activation='sigmoid'),
                layers.Reshape((28, 28))
                ])
            def call(self, x):
                encoded = self.encoder(x)
                decoded = self.decoder(encoded)
                return decoded
        
        autoencoder = Autoencoder(latent_dim)
        
        autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
        autoencoder.fit(x_train_01, x_train_01,
                epochs=10,
                shuffle=True,
                validation_data=(x_test_01, x_test_01))
        
        x_train_01, x_test_01 = autoencoder.encoder(x_train_01).numpy(), autoencoder.encoder(x_test_01).numpy()
        #Rescale for Angle Embedding
        x_train_01, x_test_01 = x_train_01 * (np.pi / 50), x_test_01 * (np.pi / 50)
        
        return x_train_01, x_test_01, y_train_01, y_test_01



In [5]:
x_train_resize256, x_test_resize256, y_train_resize256, y_test_resize256 = data_load_and_process(classes = classes, feature_reduction = 'resize256', label01 = False)
x_train_pca8, x_test_pca8, y_train_pca8, y_test_pca8 = data_load_and_process(classes = classes, feature_reduction = 'pca8', label01 = False)
x_train_autoencoder8, x_test_autoencoder8, y_train_autoencoder8, y_test_autoencoder8 = data_load_and_process(classes = classes, feature_reduction = 'autoencoder8', label01 = False)

Train on 12665 samples, validate on 2115 samples
Epoch 1/10
12665/12665 [==============================] - 1s 61us/sample - loss: 0.0637 - val_loss: 0.0342
Epoch 2/10
12665/12665 [==============================] - 1s 41us/sample - loss: 0.0296 - val_loss: 0.0267
Epoch 3/10
12665/12665 [==============================] - 1s 41us/sample - loss: 0.0256 - val_loss: 0.0241
Epoch 4/10
12665/12665 [==============================] - 1s 40us/sample - loss: 0.0230 - val_loss: 0.0218
Epoch 5/10
12665/12665 [==============================] - 1s 41us/sample - loss: 0.0212 - val_loss: 0.0205
Epoch 6/10
12665/12665 [==============================] - 1s 40us/sample - loss: 0.0203 - val_loss: 0.0197
Epoch 7/10
12665/12665 [==============================] - 1s 41us/sample - loss: 0.0196 - val_loss: 0.0191
Epoch 8/10
12665/12665 [==============================] - 0s 38us/sample - loss: 0.0191 - val_loss: 0.0186
Epoch 9/10
12665/12665 [==============================] - 0s 39us/sample - loss: 0.0188 - val_l


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



### 2. Hierarchical Quantum Circuits (Tensor Tree Networks)

In [68]:
def qubit_encoding(vector):
    for i in range(num_wires):
        qml.RY(vector[i] * 2, wires = i)


In [69]:
def U_block_t(params, wires):
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[0], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])

def U_block_b(params, wires):
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[0], wires = wires[1])
    qml.CNOT(wires = [wires[1], wires[0]])

In [71]:
num_wires = 8
dev = qml.device('default.qubit', wires = num_wires)

@qml.qnode(dev)
def variational_circuit(params, features):
    
    qubit_encoding(features)
    
    #MERA circuit
    #D blocks on (1,2), (3,4), (5,6), (7,8)    
    U_block_t(params[:2],wires=[1,2])
    U_block_b(params[2:4],wires=[3,4])
    U_block_t(params[4:6],wires=[5,6])
    #U_block_t(params[6:8],wires=[7,8])

    #U blocks on (0,1), (2,3), (4,5), (6,7), (8,9)
    U_block_t(params[6:8],wires=[0,1])
    U_block_d(params[8:10],wires=[2,3])
    U_block_t(params[10:12],wires=[4,5])
    U_block_d(params[12:14],wires=[6,7])
    #U_block_d(params[:2],wires=[1,2])
    
    #D in the middle
    U_block_t(params[14:16],wires=[2,5])
    
    #Two U's
    U_block_t(params[16:18],wires=[1,2])
    U_block_d(params[18:20],wires=[5,6])
    
    #Last U
    U_block_t(params[20:22],wires=[2,5])
    
    return qml.expval(qml.PauliZ(5))

In [73]:
def square_loss(labels, predictions):
    loss = 0
    for l,p in zip(labels, predictions):
        loss = loss + (l - p) ** 2
    return loss / len(labels)

def cost(params, X, Y):
    predictions = [variational_circuit(params, x) for x in X]
    return square_loss(Y, predictions)

In [75]:
steps = 50
batch_size = 20
num_params = 22

def circuit_training():
    opt = qml.NesterovMomentumOptimizer(0.5)
    params = np.random.randn(num_params)
    
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        params, cost_new = opt.step_and_cost(lambda p: cost(p, X_batch, Y_batch), params)
        if it % 10 == 0:
            print("Steps: ", it, " Cost: ", cost_new)
    
    return params